In [1]:
import os
import urllib
base_url = "https://resources.lendingclub.com/"
url_files = [
    "LoanStats_2019Q1.csv.zip",
    "LoanStats_2019Q2.csv.zip",
    "LoanStats_2019Q3.csv.zip",
    "LoanStats_2019Q4.csv.zip",
    "LoanStats_2020Q1.csv.zip",
]
for url_file in url_files:
    if not os.path.isfile(url_file):
        print("downloading", url_file)
        urllib.request.urlretrieve(base_url+url_file, url_file)
        print("\tdone")
    else:
        print("already has", url_file)

downloading LoanStats_2019Q1.csv.zip
	done
downloading LoanStats_2019Q2.csv.zip
	done
downloading LoanStats_2019Q3.csv.zip
	done
downloading LoanStats_2019Q4.csv.zip
	done
downloading LoanStats_2020Q1.csv.zip
	done


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership", "annual_inc", 
    "verification_status", "pymnt_plan", "dti", "delinq_2yrs", 
    "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "total_acc", 
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt", 
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", 
    "total_rec_late_fee", "recoveries", "collection_recovery_fee", 
    "last_pymnt_amnt", "collections_12_mths_ex_med", "policy_code", 
    "application_type", "acc_now_delinq", "tot_coll_amt", "tot_cur_bal", 
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", 
    "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", 
    "open_rv_24m", "max_bal_bc", "all_util", "total_rev_hi_lim", "inq_fi", 
    "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", 
    "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", "delinq_amnt", 
    "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", 
    "mo_sin_rcnt_tl", "mort_acc", "mths_since_recent_bc", 
    "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl", 
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", 
    "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m", 
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", 
    "pub_rec_bankruptcies", "tax_liens", "tot_hi_cred_lim", 
    "total_bal_ex_mort", "total_bc_limit", "total_il_high_credit_limit", 
    "hardship_flag", "debt_settlement_flag",
    "loan_status"
]

target = "loan_status"

In [4]:
# Load the data
df1 = pd.read_csv(Path('LoanStats_2019Q1.csv.zip'), skiprows=1)[:-2]
df2 = pd.read_csv(Path('LoanStats_2019Q2.csv.zip'), skiprows=1)[:-2]
df3 = pd.read_csv(Path('LoanStats_2019Q3.csv.zip'), skiprows=1)[:-2]
df4 = pd.read_csv(Path('LoanStats_2019Q4.csv.zip'), skiprows=1)[:-2]

df = pd.concat([df1, df2, df3, df4]).loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)


low_risk_rows = df[df[target] == 'low_risk']
high_risk_rows = df[df[target] == 'high_risk']

#df = pd.concat([low_risk_rows, high_risk_rows.sample(n=len(low_risk_rows), replace=True)])
df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=42), high_risk_rows])
df = df.reset_index(drop=True)
df = df.rename({target:'target'}, axis="columns")
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,15000.0,0.1612,528.25,RENT,175000.0,Not Verified,n,11.29,0.0,0.0,...,50.0,0.0,0.0,90411.0,33519.0,51200.0,28679.0,N,N,low_risk
1,25000.0,0.0819,509.19,MORTGAGE,104000.0,Not Verified,n,24.26,0.0,0.0,...,0.0,0.0,0.0,334698.0,103398.0,22200.0,90398.0,N,N,low_risk
2,24000.0,0.0702,741.27,MORTGAGE,330000.0,Not Verified,n,15.99,0.0,2.0,...,10.0,0.0,0.0,1038896.0,145218.0,109300.0,161262.0,N,N,low_risk
3,25525.0,0.1308,581.82,MORTGAGE,103000.0,Verified,n,32.44,0.0,0.0,...,80.0,0.0,0.0,192416.0,54264.0,11400.0,63216.0,N,N,low_risk
4,15000.0,0.2969,634.23,RENT,134000.0,Not Verified,n,27.20,0.0,0.0,...,100.0,0.0,0.0,136000.0,116844.0,48500.0,85000.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12973,25000.0,0.1171,552.46,RENT,120000.0,Not Verified,n,8.94,0.0,1.0,...,0.0,0.0,0.0,77424.0,32017.0,51500.0,10000.0,N,N,high_risk
12974,14400.0,0.1430,494.26,MORTGAGE,75000.0,Not Verified,n,22.50,0.0,0.0,...,50.0,0.0,0.0,305090.0,36244.0,15100.0,53974.0,N,N,high_risk
12975,15000.0,0.2305,423.29,RENT,83000.0,Source Verified,n,17.58,0.0,1.0,...,80.0,0.0,0.0,146249.0,133217.0,14900.0,127049.0,N,N,high_risk
12976,40000.0,0.0646,1225.24,MORTGAGE,92108.0,Verified,n,17.29,0.0,0.0,...,16.7,0.0,0.0,405077.0,75492.0,116300.0,12261.0,N,N,high_risk


In [5]:
df.to_csv('../2019loans.csv', index=False)

In [6]:
# Load the data
validate_df = pd.read_csv(Path('LoanStats_2020Q1.csv.zip'), skiprows=1)[:-2]
validate_df = validate_df.loc[:, columns].copy()

# Drop the null columns where all values are null
validate_df = validate_df.dropna(axis='columns', how='all')

# Drop the null rows
validate_df = validate_df.dropna()

# Remove the `Issued` loan status
issued_mask = validate_df[target] != 'Issued'
validate_df = validate_df.loc[issued_mask]

# convert interest rate to numerical
validate_df['int_rate'] = validate_df['int_rate'].str.replace('%', '')
validate_df['int_rate'] = validate_df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = dict.fromkeys(['Current', 'Fully Paid'], 'low_risk') 
validate_df = validate_df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period', 'Charged Off'], 'high_risk')    
validate_df = validate_df.replace(x)

low_risk_rows = validate_df[validate_df[target] == 'low_risk']
high_risk_rows = validate_df[validate_df[target] == 'high_risk']

validate_df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=37), high_risk_rows])
validate_df = validate_df.reset_index(drop=True)
validate_df = validate_df.rename({target:'target'}, axis="columns")
validate_df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,8000.0,0.1171,176.79,RENT,35000.0,Not Verified,n,33.78,0.0,1.0,...,0.0,0.0,0.0,59365.0,27495.0,12000.0,44565.0,N,N,low_risk
1,10000.0,0.1524,347.83,RENT,63000.0,Source Verified,n,28.27,0.0,0.0,...,100.0,0.0,0.0,62218.0,38868.0,24500.0,37718.0,N,N,low_risk
2,11900.0,0.1102,389.71,MORTGAGE,85000.0,Not Verified,n,38.50,0.0,0.0,...,83.3,0.0,0.0,228511.0,97206.0,27500.0,103301.0,N,N,low_risk
3,20000.0,0.1102,654.97,MORTGAGE,250000.0,Source Verified,n,9.50,0.0,2.0,...,33.3,0.0,0.0,298221.0,74429.0,57200.0,52521.0,N,N,low_risk
4,10000.0,0.1695,248.26,RENT,42000.0,Source Verified,n,21.54,0.0,1.0,...,0.0,0.0,0.0,39525.0,28202.0,7300.0,29425.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7609,24000.0,0.0756,747.22,RENT,50000.0,Not Verified,n,26.81,0.0,0.0,...,0.0,0.0,0.0,62375.0,18928.0,13300.0,30775.0,N,N,high_risk
7610,10000.0,0.2305,387.36,RENT,33000.0,Verified,n,38.51,0.0,2.0,...,0.0,0.0,0.0,43250.0,33022.0,8500.0,29550.0,N,N,high_risk
7611,8000.0,0.1862,205.86,RENT,38000.0,Source Verified,n,16.36,0.0,1.0,...,0.0,1.0,0.0,31357.0,19595.0,1500.0,9657.0,N,N,high_risk
7612,30000.0,0.2055,1123.34,RENT,180000.0,Source Verified,n,12.06,0.0,0.0,...,33.3,0.0,0.0,218686.0,209389.0,5600.0,209986.0,N,N,high_risk


In [7]:
validate_df.to_csv('../2020Q1loans.csv', index=False)